<p style="color: red; font-weight: bold; font-size: xx-large; text-align: center;" align="center">
    'CARA' (COVID Airborne Risk Assessment)
</p>

<p style="color: blue; font-weight: bold; font-size: x-large; text-align: center;" align="center">
Airborne Transmission of SARS-CoV-2
</p>

In [2]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import typing

In [9]:
import cara.models


def prepare_model(volume, n_infected=1, n_exposed=10, mask='Type I') -> cara.models.Model:
    """
    Transform configurable values into a cara model instance.
    
    """
    model = cara.models.Model(
        room=cara.models.Room(volume=volume),
        ventilation=cara.models.Ventilation(),
        infected=cara.models.InfectedPerson(
            virus=cara.models.Virus.types['SARS_CoV_2'],
            present_times=((0, 4), (5, 8)),
            mask=cara.models.Mask.types[mask],
            activity=cara.models.Activity.types['Light exercise'],
            expiration=cara.models.Expiration.types['Unmodulated Vocalization'],
        ),
        infected_occupants=n_infected,
        exposed_occupants=n_exposed,
        exposed_activity=cara.models.Activity.types['Light exercise'],
    )
    return model

ModuleNotFoundError: No module named 'cara'

In [10]:
# Setup our plotting environment.
plt.interactive(False)
fig_concentration_over_time = plt.figure()

In [11]:
# Define some useful widget machinery.

def collapsible(widgets_to_collapse: typing.List, title: str, start_collapsed=True):
    collapsed = widgets.Accordion([widgets.VBox(widgets_to_collapse)])
    collapsed.set_title(0, title)
    if start_collapsed:
        collapsed.selected_index = None
    return collapsed


def widget_group(label_widget_pairs):
    labels, widgets_ = zip(*label_widget_pairs)    
    labels_w = widgets.VBox(labels)
    widgets_w = widgets.VBox(widgets_)
    return widgets.HBox([labels_w, widgets_w])

In [12]:
room_volume = widgets.IntSlider(value=75, min=10, max=150)
mask_used = widgets.Checkbox(value=True, description='Mask worn')

collapsible(
    [widget_group(
        [[widgets.Label('Room volume'), room_volume]]
    )],
    title='Specification of workplace', start_collapsed=False,
)

Accordion(children=(VBox(children=(HBox(children=(VBox(children=(Label(value='Room volume'),)), VBox(children=…

In [13]:
ventilation_widgets = {
    'Natural': widgets.Label('Currently hard-coded to 514.74 / volume'),
    'other': widgets.Label('Not yet implemented.')
}
for name, widget in ventilation_widgets.items():
    widget.layout.visible = False

ventilation_w = widgets.ToggleButtons(
    options=['Natural'],  # cara.models.Ventilation.types.keys(),
)
def toggle_ventilation(value):
    for name, widget in ventilation_widgets.items():
        widget.layout.display = 'none'
    other = ventilation_widgets['other']
    widget = ventilation_widgets.get(value, other)
    widget.layout.visible = True
    widget.layout.display = 'block'

ventilation_w.observe(lambda event: toggle_ventilation(event['new']), 'value')
toggle_ventilation(ventilation_w.value)


collapsible(
    [widget_group([[widgets.Label('Ventilation type'), ventilation_w]])]
    + list(ventilation_widgets.values()),
    title='Ventilation scheme'
)

Accordion(children=(VBox(children=(HBox(children=(VBox(children=(Label(value='Ventilation type'),)), VBox(chil…

In [14]:
import matplotlib.pyplot as plt

line = None
# plt.ioff()
# fig = plt.figure()
fig = fig_concentration_over_time

def plot_concentrations(_):
    global line
    model = prepare_model(room_volume.value)

    ts = np.arange(0, 10., 0.01)
    concentration = [model.concentration(t) for t in ts]

    ax = fig.gca()
    
    plt.text(0.5, 0.9, 'Without masks & window open', transform=ax.transAxes, ha='center')
    if line is None:
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        [line] = plt.plot(ts, concentration)
        ax.set_xlabel('Time (hours)')
        ax.set_ylabel('Concentration ($q/m^3$)')
        plt.title('Concentration of infectious quanta aerosols')
        
        ax.set_ymargin(0.2)
        ax.set_ylim(bottom=0)
    else:
        line.set_data(ts, concentration)
        ax.relim()
        ax.autoscale_view()
        
    plt.draw()

#     print(f'Probability of infection: {np.round(model.["P"], 1)}')
#     print(f'Expected number of new cases: {prepared["R0"]}')
    

# widgets.interact(
#     plot_concentrations,
#     volume=room_volume,
#     n_exposed=widgets.IntSlider(value=10, min=0, max=25),
#     n_infected=widgets.IntSlider(value=1, min=1, max=5),
# );


for observable in [room_volume]:
    observable.observe(plot_concentrations)

plot_concentrations(1)

fig.canvas.toolbar_visible = True
fig.canvas.toolbar.collapsed = True
fig.canvas.footer_visible = False
fig.canvas.header_visible = False


collapsible([
    widgets.HBox([
        fig.canvas,
        # text_report,
    ])
], 'Report', start_collapsed=False)


NameError: name 'prepare_model' is not defined